In [76]:
import os
import json
import pandas as pd
import numpy as np
import csv
import gdal

In [2]:
"""path = 'data/sample.tsv'
path = 'data/twex.tsv'
nb_fields = 20"""

"path = 'data/sample.tsv'\npath = 'data/twex.tsv'\nnb_fields = 20"

In [3]:
"""ids = ""
with open("data/schema.txt", "r") as schema:
    content = schema.readlines()
    for line in content:
        if(ids == ""):
            ids = ids + (line.split("    ")[1])
        else:
            ids = ids + " " + (line.split("    ")[1])
idsList = list(ids.split(" "))
print(idsList)"""

'ids = ""\nwith open("data/schema.txt", "r") as schema:\n    content = schema.readlines()\n    for line in content:\n        if(ids == ""):\n            ids = ids + (line.split("    ")[1])\n        else:\n            ids = ids + " " + (line.split("    ")[1])\nidsList = list(ids.split(" "))\nprint(idsList)'

In [4]:
'''cleanedPath = "data/output.tsv"
values = []
with open(path) as tsvfile:
    content = tsvfile.readlines()
    with open(path, "w") as outputfile:
        for i in range(len(content)):
            if i%10000 == 0:
                print("line:", i)
            line = content[i]
            if(len(line.split("\t")) == nb_fields):
                values.append(line.split("\t"))
        tsvfile.close()'''

'cleanedPath = "data/output.tsv"\nvalues = []\nwith open(path) as tsvfile:\n    content = tsvfile.readlines()\n    with open(path, "w") as outputfile:\n        for i in range(len(content)):\n            if i%10000 == 0:\n                print("line:", i)\n            line = content[i]\n            if(len(line.split("\t")) == nb_fields):\n                values.append(line.split("\t"))\n        tsvfile.close()'

In [5]:
"""cleanedPath = "data/cleaned.tsv"
with open(path) as infile:
    with open(cleanedPath, "w") as outputfile:
        i=0
        for line in infile:
            i = i+1
            if i%200000 == 0:
                print("line:", i)
            line_array = line.split("\t")
            if(len(line_array) == nb_fields):
                if line_array[4] != "\\N":
                    outputfile.write(line_array[0])
                    outputfile.write("\t")
                    outputfile.write(line_array[2])
                    outputfile.write("\t")
                    outputfile.write(line_array[4])
                    outputfile.write("\t")
                    outputfile.write(line_array[5])
                    outputfile.write("\n")
        outputfile.close()
    infile.close()"""

'cleanedPath = "data/cleaned.tsv"\nwith open(path) as infile:\n    with open(cleanedPath, "w") as outputfile:\n        i=0\n        for line in infile:\n            i = i+1\n            if i%200000 == 0:\n                print("line:", i)\n            line_array = line.split("\t")\n            if(len(line_array) == nb_fields):\n                if line_array[4] != "\\N":\n                    outputfile.write(line_array[0])\n                    outputfile.write("\t")\n                    outputfile.write(line_array[2])\n                    outputfile.write("\t")\n                    outputfile.write(line_array[4])\n                    outputfile.write("\t")\n                    outputfile.write(line_array[5])\n                    outputfile.write("\n")\n        outputfile.close()\n    infile.close()'

In [6]:
data = pd.DataFrame.from_csv("data/cleaned.tsv", sep="\t", header=None)

In [7]:
data.columns=["date","longitude","latitude"]
data.head()

,date,longitude,latitude
0,,,
9514097914,2010-02-23 05:55:51,7.43926,46.9489
9514846412,2010-02-23 06:22:40,8.53781,47.3678
9516574359,2010-02-23 07:34:25,6.13396,46.1951
9516952605,2010-02-23 07:51:47,8.81749,47.2288
9517198943,2010-02-23 08:02:57,6.63254,46.5199


In [8]:
localisation = data[["latitude","longitude"]]
localisation.head()

,latitude,longitude
0,,
9514097914,46.9489,7.43926
9514846412,47.3678,8.53781
9516574359,46.1951,6.13396
9516952605,47.2288,8.81749
9517198943,46.5199,6.63254


In [42]:
dataset = gdal.Open( "data/dilatationNDBI.tif" )


print('Driver: ', dataset.GetDriver().ShortName,'/', \
      dataset.GetDriver().LongName)
print( 'Size is ',dataset.RasterXSize,'x',dataset.RasterYSize, \
      'x',dataset.RasterCount)
print( 'Projection is ',dataset.GetProjection())
geotransform = dataset.GetGeoTransform()
if not geotransform is None:
    print( 'Origin = (',geotransform[0], ',',geotransform[3],')')
    print( 'Pixel Size = (',geotransform[1], ',',geotransform[5],')')

Driver:  GTiff / GeoTIFF
Size is  5261 x 2331 x 1
Projection is  GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]
Origin = ( 5.863303859448117 , 47.887391165843454 )
Pixel Size = ( 0.0008983152841195215 , -0.0008983152841195215 )


In [71]:
srcband = dataset.GetRasterBand(1)
#stats = srcband.GetStatistics( True, True )

mask = srcband.ReadAsArray(0, 0, srcband.XSize, srcband.YSize)
print(mask.shape)
print(srcband.XSize)

topleftX = geotransform[0]
topleftY = geotransform[3]
pixelsizeX = geotransform[1]
pixelsizeY = geotransform[5]

bottomrightX = topleftX+(pixelsizeX*dataset.RasterXSize)
bottomrightY = topleftY+(pixelsizeY*dataset.RasterYSize)


print("topLeft:", topleftX, topleftY)
print("bottomRight:", bottomrightX, bottomrightY)




(2331, 5261)
5261
topLeft: 5.863303859448117 47.887391165843454
bottomRight: 10.589340569200921 45.79341823856085


In [79]:
np.save("data/urban_mask.npy", mask)

In [86]:
mask = 'data/urban_mask.npy'
data_path = 'data/cleaned.tsv'
with open(data_path) as data:
    i = 0
    for line in data:
        if i ==0:
            i = i+1
            print(line)
            print(line.split('\t')[2:])

9514097914	2010-02-23 05:55:51	7.43926	46.9489

['7.43926', '46.9489\n']


In [ ]:
def getValue(long, lat): 
    xPos = int(1.0 * (long - topleftX) / (bottomrightX - topleftX) * srcband.XSize)
    yPos = srcband.YSize-1  - int(1.0 * (lat - bottomrightY) / (topleftY - bottomrightY) * srcband.YSize)
    if (xPos >= srcband.XSize or xPos < 0):
        return False
    if (yPos >= srcband.YSize or yPos < 0):
        return False
    return mask[yPos, xPos] == 0

def getValueFrame(row): 
    long = row["longitude"]
    lat = row["latitude"]
    xPos = int(1.0 * (long - topleftX) / (bottomrightX - topleftX) * srcband.XSize)
    yPos = srcband.YSize-1  - int(1.0 * (lat - bottomrightY) / (topleftY - bottomrightY) * srcband.YSize)
    if (xPos >= srcband.XSize or xPos < 0):
        return False
    if (yPos >= srcband.YSize or yPos < 0):
        return False
    return mask[yPos, xPos] == 0


In [72]:
if getValue(8,46.961):
    print("Yes")
else:
    print("No")

No


In [73]:
newdata = data
newdata["used"] = False
newdata.head()

,date,longitude,latitude,urban,used
0,,,,,
9514097914,2010-02-23 05:55:51,7.43926,46.9489,False,False
9514846412,2010-02-23 06:22:40,8.53781,47.3678,False,False
9516574359,2010-02-23 07:34:25,6.13396,46.1951,False,False
9516952605,2010-02-23 07:51:47,8.81749,47.2288,False,False
9517198943,2010-02-23 08:02:57,6.63254,46.5199,False,False


In [74]:
newdata['used'] = newdata.apply (lambda row: getValueFrame(row),axis=1)
    
newdata.head()

,date,longitude,latitude,urban,used
0,,,,,
9514097914,2010-02-23 05:55:51,7.43926,46.9489,False,False
9514846412,2010-02-23 06:22:40,8.53781,47.3678,False,False
9516574359,2010-02-23 07:34:25,6.13396,46.1951,False,False
9516952605,2010-02-23 07:51:47,8.81749,47.2288,False,False
9517198943,2010-02-23 08:02:57,6.63254,46.5199,False,False


In [89]:
newdata[newdata['used'] == True].count()/newdata.count()

date         0.149538
longitude    0.149538
latitude     0.149538
urban        0.149538
used         0.149538
dtype: float64

In [91]:
urbanized = newdata[["date","longitude","latitude","used"]]
urbanized.head()

,date,longitude,latitude,used
0,,,,
9514097914,2010-02-23 05:55:51,7.43926,46.9489,False
9514846412,2010-02-23 06:22:40,8.53781,47.3678,False
9516574359,2010-02-23 07:34:25,6.13396,46.1951,False
9516952605,2010-02-23 07:51:47,8.81749,47.2288,False
9517198943,2010-02-23 08:02:57,6.63254,46.5199,False


In [93]:
urbanized.to_csv("data/cleaned_with_urbanity.tsv", sep="\t")

In [94]:
urbanized[urbanized["used"] == True].to_csv("data/urban_jjmayor.tsv", sep=",")

In [96]:
non_urban = urbanized[urbanized["used"] == True].copy()
non_urban.head()

,date,longitude,latitude,used
0,,,,
9564875001,2010-02-24 06:26:53,7.44237,46.8958,True
9571982009,2010-02-24 11:31:52,8.38014,47.3679,True
9575552318,2010-02-24 13:28:52,8.06674,46.3913,True
9575623646,2010-02-24 13:30:51,8.06763,46.3910,True
9587557928,2010-02-24 18:45:38,8.77847,47.2034,True


In [ ]:
"""from sklearn.cluster import KMeans
kmeans = KMeans(init='k-means++', n_clusters=100, n_init=1, max_iter=10, n_jobs = 4)
kmeans.fit(localisation)
print("It's over!")

import matplotlib.pyplot as plt
plt.figure(1)

# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1])
plt.title('K-means clustering on the digits dataset (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(45, 49)
plt.ylim(5, 11)
plt.show()"""

In [105]:
from sklearn.cluster import KMeans
kmeans = KMeans(init='k-means++', n_clusters=1000, n_init=1, max_iter=10, n_jobs = 4)
kmeans.fit(non_urban[["longitude","latitude"]])
print("It's over!")


/home/nathan/anaconda3/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


It's over!


In [106]:

import matplotlib.pyplot as plt
plt.figure(1)

# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.xlim(45, 49)
plt.ylim(5, 11)
plt.scatter(centroids[:, 1], centroids[:, 0])
plt.show()

In [107]:
centroids

array([[  8.80982957,  47.08625854],
       [  6.66646296,  46.62231439],
       [  9.82554179,  46.79613678],
       ..., 
       [  7.35993138,  46.61889328],
       [  9.72760808,  45.86711647],
       [  7.4876492 ,  46.45049045]])

In [135]:
np_array = non_urban[["longitude","latitude"]].as_matrix()
np_array.shape

array([677,  35, 564, ..., 269, 317, 219], dtype=int32)

In [156]:
step = 100000
predictions = [[]]

for i in range(int(np_array.shape[0]/step)):
    if i == 0:
        predictions = kmeans.predict(np_array[i*step:(i+1)*step])
    else :
        predictions = np.concatenate([predictions, kmeans.predict(np_array[i*step:(i+1)*step])])

In [157]:
i = i+1
predictions = np.concatenate([predictions, kmeans.predict(np_array[i*step:(i+1)*step])])

In [166]:
predictions.shape

(1860033,)

In [168]:
predictions

array([677,  35, 564, ..., 490, 701, 701], dtype=int32)

In [169]:
np.savetxt("data/cluster_assign.csv", predictions.astype(int), fmt='%d')

In [171]:
non_urban["cluster"] = predictions
non_urban.head()

,date,longitude,latitude,used,cluster
0,,,,,
9564875001,2010-02-24 06:26:53,7.44237,46.8958,True,677
9571982009,2010-02-24 11:31:52,8.38014,47.3679,True,35
9575552318,2010-02-24 13:28:52,8.06674,46.3913,True,564
9575623646,2010-02-24 13:30:51,8.06763,46.3910,True,564
9587557928,2010-02-24 18:45:38,8.77847,47.2034,True,119


In [172]:
non_urban.to_csv("data/cleaned_non_urban_clusterized.tsv", sep="\t")